In [ ]:
import pandas as pd
import numpy as np
import ast
import geopy.geocoders
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

načtení souboru, odmazání sloupce type, přejmenování názvů sloupců

In [ ]:
trafficDelaysDf = pd.read_csv('C:\Users\Misulka\Desktop\Czechitas\Digitalni_akademie\Python\Projekt/plynulostVSE.csv')

In [ ]:
trafficDelaysDf = trafficDelaysDf.drop(columns = ['type'])
trafficDelaysDf.columns = trafficDelaysDf.columns.str.replace('properties.','')
trafficDelaysDf= trafficDelaysDf.rename(columns={'geometry.type':'coordType'})
trafficDelaysDf= trafficDelaysDf.rename(columns={'geometry.coordinates':'coordinates'})

převod souřadnic ze str na seznam seznamů

In [ ]:
coordList = []
for i in trafficDelaysDf.index:
    nestList = ast.literal_eval(trafficDelaysDf["coordinates"][i])
    coordList.append(nestList)
trafficDelaysDf2 = trafficDelaysDf.copy() #tvorba kopie původní tabulky
trafficDelaysDf2['coordinates']=coordList #náhrada původního sloupce coordinates

rozšíření tabulky o řádky s jednotlivými souřadnicemi

In [ ]:
newList = []
allrows =trafficDelaysDf2.values.tolist()#prevod hodnot v tabulce na seznam allrows
for line in allrows:
    newList += [line[:-1] + [coor] for coor in line[-1]] #kod od Martina, list comprehension misto explode

newListMulti = [] # další zanoření pro multilinestring
for line in newList:
    if 'MultiLineString' in line:
        newListMulti += [line[:-1] + [coor] for coor in line[-1]]
    else:
        newListMulti.append(line)

dataFrame3 ze seznamu

In [ ]:
trafficDelaysDf3= pd.DataFrame(newListMulti)

#výpis hodnot sloupce coordinates pro multilinestring
trafficDelaysDf3[18][trafficDelaysDf3[17]=='MultiLineString'].head()


přiřazení původních názvů sloupců a kontrola

In [ ]:
trafficDelaysDf3.columns =trafficDelaysDf.columns

In [ ]:
trafficDelaysDf3.shape


kontrolní tabulka před rozšířením o multilinestring

In [ ]:
trafficDelaysDf4= pd.DataFrame(newList)
trafficDelaysDf4.shape

uložení tabulky df3 do csv v kompletní a zkrácené verzi

In [ ]:
trafficDelaysDf3.to_csv('trafficDf3.csv')
trafficDelaysDf3.head(1000).to_csv('trafficDf_1000.csv')

ODTUD PŘÍPRAVA PRO přiřazení názvů ulic podle souřadnic

In [ ]:
geopy.geocoders.options.default_user_agent = 'Mozilla/5.0'
geolocator = Nominatim()

In [ ]:
type(trafficDelaysDf3['coordinates'][0][0])

načtení stručné původní tabulky s uuid a coordinates

In [ ]:
trafficDelaysCoord = pd.read_csv("coord.csv")

nahrazení NaN v původní tabulce "ničím"

In [ ]:
trafficDelaysDf["street"].fillna('', inplace=True)

In [ ]:
trafficDelaysDf["street"].head()

tabulka streetNaN z původního df a výpis sloupce uuid ze stručné tabulky pro podmínku kde street je prázdná

In [ ]:
streetNaN = trafficDelaysDf["uuid"][trafficDelaysDf["street"]==""]

In [ ]:
trafficDelaysCoord["uuid"].isin(streetNaN)


tabulka prázdných ulic

In [ ]:
trafficDelaysCoordNaN = trafficDelaysCoord[trafficDelaysCoord["uuid"].isin(streetNaN)]
trafficDelaysCoordNaN.head()
#trafficDelaysCoordNaN.to_csv('missing.csv')

přiřazení názvů

In [ ]:
streetList = []
for rows in trafficDelaysCoordNaN['coordinates']:
    souradnice = str(row[1])+','+ str(row[0])
    
    #geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
    #df['location'] = df['name'].apply(geocode)
    
    location = geolocator.geocode(souradnice, language = "cz")
    nazev = location.raw['display_name']
    ulice = nazev.split(",")
    if len(ulice) == 9:
        streetList.append(ulice[1])
    elif len(ulice) == 8:
        streetList.append(ulice[0])
    else:
        streetList.append(ulice)